# Variable dependiente: Acceso a Educación Superior (multiclase)

## 1. Cargar data

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as  pd, numpy as np
import variables as vb

In [3]:
# Cargar datos
path = r'..\..\output\data_preprocess\dfs_0_acc.csv'
data_original = pd.read_csv( path )

In [4]:
data_original.shape

(875, 396)

## 2. Scale only numeric vars

In [5]:
# https://stackoverflow.com/questions/38420847/apply-standardscaler-to-parts-of-a-data-set

from sklearn.preprocessing import StandardScaler

data = data_original.copy()

numeric_vars = [col for col in data.columns if col in vb.num_vars ]

cols = data[ numeric_vars ]
scaler = StandardScaler().fit( cols.values )
cols = scaler.transform( cols.values )

data[ numeric_vars ] = cols

## 3. Split variables

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
data[ 'e_acceso' ] = data[ 'e_acceso' ].replace([1.0], 0.0)
data[ 'e_acceso' ] = data[ 'e_acceso' ].replace([2.0], 1.0)
data[ 'e_acceso' ] = data[ 'e_acceso' ].replace([3.0], 2.0)

In [8]:
data[ 'e_acceso' ].value_counts()

0.0    469
2.0    245
1.0    161
Name: e_acceso, dtype: int64

In [9]:
dep_var = [ 'e_acceso' ]
pred_vars = [col for col in data.columns if col not in vb.dep_vars and col not in dep_var ]
x_train, x_test, y_train, y_test = train_test_split( data[ pred_vars ], data[ 'e_acceso' ], test_size = 0.20 )

## 4. Logistic Regression

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, log_loss, roc_auc_score, f1_score
from sklearn.metrics import classification_report

In [11]:
%%time

# Implementing the model
lg_model = LogisticRegression( max_iter = 10000, multi_class='multinomial' ).fit( x_train, y_train )

# Predict over test set
y_lg_pred_class = lg_model.predict( x_test )
y_lg_pred_prob = lg_model.predict_proba( x_test )

Wall time: 517 ms


In [12]:
# Calculating metrics
columns = [ 'no', 'tec', 'uni' ]
lg_report = classification_report(y_test, y_lg_pred_class, target_names = columns, output_dict = True )

lg_no_precision = lg_report[ 'no' ][ 'precision' ]
lg_no_recall = lg_report[ 'no' ][ 'recall' ]
lg_no_f1_score = lg_report[ 'no' ][ 'f1-score' ]

lg_tec_precision = lg_report[ 'tec' ][ 'precision' ]
lg_tec_recall = lg_report[ 'tec' ][ 'recall' ]
lg_tec_f1_score = lg_report[ 'tec' ][ 'f1-score' ]

lg_uni_precision = lg_report[ 'uni' ][ 'precision' ]
lg_uni_recall = lg_report[ 'uni' ][ 'recall' ]
lg_uni_f1_score = lg_report[ 'uni' ][ 'f1-score' ]

accuracy_lg = accuracy_score( y_test, y_lg_pred_class )
roc_auc_lg = roc_auc_score( y_test, y_lg_pred_prob, multi_class = "ovr", average = "weighted" )

In [13]:
print( classification_report(y_test, y_lg_pred_class, target_names = columns ) )

              precision    recall  f1-score   support

          no       0.51      0.51      0.51        86
         tec       0.18      0.22      0.20        27
         uni       0.43      0.37      0.40        62

    accuracy                           0.42       175
   macro avg       0.37      0.37      0.37       175
weighted avg       0.43      0.42      0.42       175



Info:

* About meaning for each indicator: https://towardsdatascience.com/comprehensive-guide-to-multiclass-classification-with-sklearn-127cc500f362
* Also that link states that ovr is a better indicator

## 5. Regularization Methods (Lasso, Ridge and Elastic Net)

In [14]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV

## 5.1. Lasso

In [15]:
%%time

# Implementing the model
lasso_model = LogisticRegressionCV( penalty = 'l1', solver = 'saga', cv = 10, random_state = 0, max_iter = 10000 ).\
                               fit( x_train, y_train )

# Predict over test set
y_lasso_pred_class = lasso_model.predict( x_test )
y_lasso_pred_prob = lasso_model.predict_proba( x_test )

Wall time: 13min 9s


In [16]:
# Calculating metrics
columns = [ 'no', 'tec', 'uni' ]
lasso_report = classification_report( y_test, y_lasso_pred_class, target_names = columns, output_dict = True )

lasso_no_precision = lasso_report[ 'no' ][ 'precision' ]
lasso_no_recall = lasso_report[ 'no' ][ 'recall' ]
lasso_no_f1_score = lasso_report[ 'no' ][ 'f1-score' ]

lasso_tec_precision = lasso_report[ 'tec' ][ 'precision' ]
lasso_tec_recall = lasso_report[ 'tec' ][ 'recall' ]
lasso_tec_f1_score = lasso_report[ 'tec' ][ 'f1-score' ]

lasso_uni_precision = lasso_report[ 'uni' ][ 'precision' ]
lasso_uni_recall = lasso_report[ 'uni' ][ 'recall' ]
lasso_uni_f1_score = lasso_report[ 'uni' ][ 'f1-score' ]

accuracy_lasso = accuracy_score( y_test, y_lasso_pred_class )
roc_auc_lasso = roc_auc_score( y_test, y_lasso_pred_prob, multi_class = "ovr", average = "weighted" )

In [17]:
print( classification_report( y_test, y_lasso_pred_class, target_names = columns ) )

              precision    recall  f1-score   support

          no       0.54      0.91      0.68        86
         tec       0.00      0.00      0.00        27
         uni       0.67      0.32      0.43        62

    accuracy                           0.56       175
   macro avg       0.40      0.41      0.37       175
weighted avg       0.50      0.56      0.49       175



## 5.2. Ridge

In [18]:
%%time

# Implementing the model
ridge_model = LogisticRegressionCV( penalty = 'l2', solver = 'saga', cv = 10, random_state = 0, max_iter = 10000 ).\
                               fit( x_train, y_train )

# Predict over test set
y_ridge_pred_class = ridge_model.predict( x_test )
y_ridge_pred_prob = ridge_model.predict_proba( x_test )

Wall time: 10min 53s


In [19]:
# Calculating metrics
columns = [ 'no', 'tec', 'uni' ]
ridge_report = classification_report( y_test, y_ridge_pred_class, target_names = columns, output_dict = True )

ridge_no_precision = ridge_report[ 'no' ][ 'precision' ]
ridge_no_recall = ridge_report[ 'no' ][ 'recall' ]
ridge_no_f1_score = ridge_report[ 'no' ][ 'f1-score' ]

ridge_tec_precision = ridge_report[ 'tec' ][ 'precision' ]
ridge_tec_recall = ridge_report[ 'tec' ][ 'recall' ]
ridge_tec_f1_score = ridge_report[ 'tec' ][ 'f1-score' ]

ridge_uni_precision = ridge_report[ 'uni' ][ 'precision' ]
ridge_uni_recall = ridge_report[ 'uni' ][ 'recall' ]
ridge_uni_f1_score = ridge_report[ 'uni' ][ 'f1-score' ]

accuracy_ridge = accuracy_score( y_test, y_ridge_pred_class )
roc_auc_ridge = roc_auc_score( y_test, y_ridge_pred_prob, multi_class = "ovr", average = "weighted" )

In [20]:
print( classification_report( y_test, y_ridge_pred_class, target_names = columns ) )

              precision    recall  f1-score   support

          no       0.52      0.90      0.66        86
         tec       0.00      0.00      0.00        27
         uni       0.62      0.26      0.36        62

    accuracy                           0.53       175
   macro avg       0.38      0.38      0.34       175
weighted avg       0.47      0.53      0.45       175



### 5.3. Elastic Net

In [21]:
%%time

# Implementing the model
elasticnet_model = LogisticRegressionCV( penalty = 'elasticnet', solver = 'saga', cv = 10, random_state = 0, l1_ratios = [ 0.5 ], max_iter = 10000 ).\
                                    fit( x_train, y_train )

# Predict over test set
y_elasticnet_pred_class = elasticnet_model.predict( x_test )
y_elasticnet_pred_prob = elasticnet_model.predict_proba( x_test )

Wall time: 27min 25s


In [22]:
# Calculating metrics
columns = [ 'no', 'tec', 'uni' ]
elasticnet_report = classification_report( y_test, y_elasticnet_pred_class, target_names = columns, output_dict = True )

elasticnet_no_precision = elasticnet_report[ 'no' ][ 'precision' ]
elasticnet_no_recall = elasticnet_report[ 'no' ][ 'recall' ]
elasticnet_no_f1_score = elasticnet_report[ 'no' ][ 'f1-score' ]

elasticnet_tec_precision = elasticnet_report[ 'tec' ][ 'precision' ]
elasticnet_tec_recall = elasticnet_report[ 'tec' ][ 'recall' ]
elasticnet_tec_f1_score = elasticnet_report[ 'tec' ][ 'f1-score' ]

elasticnet_uni_precision = elasticnet_report[ 'uni' ][ 'precision' ]
elasticnet_uni_recall = elasticnet_report[ 'uni' ][ 'recall' ]
elasticnet_uni_f1_score = elasticnet_report[ 'uni' ][ 'f1-score' ]

accuracy_elasticnet = accuracy_score( y_test, y_elasticnet_pred_class )
roc_auc_elasticnet = roc_auc_score( y_test, y_elasticnet_pred_prob, multi_class = "ovr", average = "weighted" )

In [23]:
print( classification_report( y_test, y_elasticnet_pred_class, target_names = columns ) )

              precision    recall  f1-score   support

          no       0.51      0.94      0.66        86
         tec       0.00      0.00      0.00        27
         uni       0.60      0.15      0.23        62

    accuracy                           0.51       175
   macro avg       0.37      0.36      0.30       175
weighted avg       0.46      0.51      0.41       175



## 6. Random Forest

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [25]:
%%time

# Set the model
rf_model = RandomForestClassifier( random_state = 0 )

# Define param grid
rf_param_grid = {
    'n_estimators': [ 500, 1000, 2000 ],
    'max_features': [ 'auto', 'sqrt', 'log2' ]
}

# Grid search
rf_search = GridSearchCV( estimator = rf_model,
                          param_grid = rf_param_grid )

# Fit to data
rf_search.fit( x_train, y_train )

# Print best params and best score
print( rf_search.best_params_ )

# Select best params
rf_max_features = rf_search.best_params_[ 'max_features' ] 
rf_n_estimators = rf_search.best_params_[ 'n_estimators' ] 

{'max_features': 'auto', 'n_estimators': 1000}
Wall time: 5min 24s


In [26]:
# Train the optimal model
rf_optimal_model = RandomForestClassifier( max_features = rf_max_features, 
                                           n_estimators = rf_n_estimators )
rf_optimal_model.fit( x_train, y_train )

# Apply over test set
y_rf_pred = rf_optimal_model.predict( x_test )

# Predict over test set
y_rf_pred_class = rf_optimal_model.predict( x_test )
y_rf_pred_prob = rf_optimal_model.predict_proba( x_test )

In [27]:
# Calculating metrics
columns = [ 'no', 'tec', 'uni' ]
rf_report = classification_report(y_test, y_rf_pred_class, target_names = columns, output_dict = True )

rf_no_precision = rf_report[ 'no' ][ 'precision' ]
rf_no_recall = rf_report[ 'no' ][ 'recall' ]
rf_no_f1_score = rf_report[ 'no' ][ 'f1-score' ]

rf_tec_precision = rf_report[ 'tec' ][ 'precision' ]
rf_tec_recall = rf_report[ 'tec' ][ 'recall' ]
rf_tec_f1_score = rf_report[ 'tec' ][ 'f1-score' ]

rf_uni_precision = rf_report[ 'uni' ][ 'precision' ]
rf_uni_recall = rf_report[ 'uni' ][ 'recall' ]
rf_uni_f1_score = rf_report[ 'uni' ][ 'f1-score' ]

accuracy_rf = accuracy_score( y_test, y_rf_pred_class )
roc_auc_rf = roc_auc_score( y_test, y_rf_pred_prob, multi_class = "ovr", average = "weighted" )

In [28]:
print( classification_report( y_test, y_rf_pred_class, target_names = columns ) )

              precision    recall  f1-score   support

          no       0.53      0.88      0.66        86
         tec       0.00      0.00      0.00        27
         uni       0.64      0.29      0.40        62

    accuracy                           0.54       175
   macro avg       0.39      0.39      0.35       175
weighted avg       0.49      0.54      0.47       175



## 7. Boosted Trees

In [29]:
from xgboost import XGBClassifier

In [30]:
%%time

# Set the model
xgb_model = XGBClassifier( use_label_encoder = False, objective = 'multi:softmax', verbosity = 0, num_class = 3 )

# Define param grid
xgb_param_grid = {
    'n_estimators': [ 500, 1000, 2000 ],
    'learning_rate': [0.1, 0.5, 1]
}

# Grid search
xgb_search = GridSearchCV( estimator = xgb_model,
                           param_grid = xgb_param_grid )

# Fit to data
xgb_search.fit( x_train, y_train )

# Print best params and best score
print( xgb_search.best_params_ )

# Select best params
xgb_learning_rate = xgb_search.best_params_[ 'learning_rate' ] 
xgb_n_estimators = xgb_search.best_params_[ 'n_estimators' ] 

{'learning_rate': 0.5, 'n_estimators': 500}
Wall time: 8min 54s


In [31]:
# Train the optimal model
xgb_optimal_model = XGBClassifier( objective = 'multi:softmax', 
                                   verbosity = 0,
                                   learning_rate = xgb_learning_rate, 
                                   n_estimators = xgb_n_estimators, 
                                   num_class = 3)
xgb_optimal_model.fit( x_train, y_train )

# Predict over test set
y_xgb_pred_class = xgb_optimal_model.predict( x_test )
y_xgb_pred_prob = xgb_optimal_model.predict_proba( x_test )

In [32]:
# Calculating metrics
columns = [ 'no', 'tec', 'uni' ]
xgb_report = classification_report(y_test, y_xgb_pred_class, target_names = columns, output_dict = True )

xgb_no_precision = xgb_report[ 'no' ][ 'precision' ]
xgb_no_recall = xgb_report[ 'no' ][ 'recall' ]
xgb_no_f1_score = xgb_report[ 'no' ][ 'f1-score' ]

xgb_tec_precision = xgb_report[ 'tec' ][ 'precision' ]
xgb_tec_recall = xgb_report[ 'tec' ][ 'recall' ]
xgb_tec_f1_score = xgb_report[ 'tec' ][ 'f1-score' ]

xgb_uni_precision = xgb_report[ 'uni' ][ 'precision' ]
xgb_uni_recall = xgb_report[ 'uni' ][ 'recall' ]
xgb_uni_f1_score = xgb_report[ 'uni' ][ 'f1-score' ]

accuracy_xgb = accuracy_score( y_test, y_xgb_pred_class )
roc_auc_xgb = roc_auc_score( y_test, y_xgb_pred_prob, multi_class = "ovr", average = "weighted" )

## 7. Resultados

In [33]:
table = np.zeros( ( 6, 11 ) )

table[ 0 ] = [ accuracy_lg, roc_auc_lg, lg_no_precision, lg_no_recall, 
               lg_no_f1_score, lg_tec_precision, lg_tec_recall, lg_tec_f1_score,
               lg_uni_precision, lg_uni_recall, lg_uni_f1_score ]

table[ 1 ] = [ accuracy_lasso, roc_auc_lasso, lasso_no_precision, lasso_no_recall, 
               lasso_no_f1_score, lasso_tec_precision, lasso_tec_recall, lasso_tec_f1_score,
               lasso_uni_precision, lasso_uni_recall, lasso_uni_f1_score ]

table[ 2 ] = [ accuracy_ridge, roc_auc_ridge, ridge_no_precision, ridge_no_recall, 
               ridge_no_f1_score, ridge_tec_precision, ridge_tec_recall, ridge_tec_f1_score,
               ridge_uni_precision, ridge_uni_recall, ridge_uni_f1_score ]

table[ 3 ] = [ accuracy_elasticnet, roc_auc_elasticnet, elasticnet_no_precision, elasticnet_no_recall, 
               elasticnet_no_f1_score, elasticnet_tec_precision, elasticnet_tec_recall, elasticnet_tec_f1_score,
               elasticnet_uni_precision, elasticnet_uni_recall, elasticnet_uni_f1_score ]

table[ 4 ] = [ accuracy_rf, roc_auc_rf, rf_no_precision, rf_no_recall, 
               rf_no_f1_score, rf_tec_precision, rf_tec_recall, rf_tec_f1_score,
               rf_uni_precision, rf_uni_recall, rf_uni_f1_score ]

table[ 5 ] = [ accuracy_xgb, roc_auc_xgb, xgb_no_precision, xgb_no_recall, 
               xgb_no_f1_score, xgb_tec_precision, xgb_tec_recall, xgb_tec_f1_score,
               xgb_uni_precision, xgb_uni_recall, xgb_uni_f1_score ]

colnames_table = [ "Overall_Accuracy", "Roc_Auc", "Ninguna_Precision", "Ninguna_Recall",
                   "Ninguna_F1_Score", "Tec_Precision", "Tec_Recall", "Tec_F1_Score",
                   "Uni_Precision", "Uni_Recall", "Uni_F1_Dcore" ]
                  
rownames_table = [ "Logistic Regression", "Lasso",
                   "Ridge", "Elastic Net",
                   "Random Forest", "Boosted Trees" ]

table_pandas = pd.DataFrame( table, columns = colnames_table )
table_pandas.index = rownames_table

table_pandas = table_pandas.round(3)
table_pandas

,Overall_Accuracy,Roc_Auc,Ninguna_Precision,Ninguna_Recall,Ninguna_F1_Score,Tec_Precision,Tec_Recall,Tec_F1_Score,Uni_Precision,Uni_Recall,Uni_F1_Dcore
Logistic Regression,0.417,0.565,0.506,0.512,0.509,0.176,0.222,0.197,0.426,0.371,0.397
Lasso,0.560,0.605,0.538,0.907,0.675,0.000,0.000,0.000,0.667,0.323,0.435
Ridge,0.531,0.607,0.517,0.895,0.655,0.000,0.000,0.000,0.615,0.258,0.364
Elastic Net,0.514,0.624,0.506,0.942,0.659,0.000,0.000,0.000,0.600,0.145,0.234
Random Forest,0.537,0.625,0.528,0.884,0.661,0.000,0.000,0.000,0.643,0.290,0.400
Boosted Trees,0.497,0.594,0.530,0.721,0.611,0.125,0.074,0.093,0.548,0.371,0.442


## 8. Feature map

In [34]:
# Random Forest
fp_randomforest = pd.Series( rf_optimal_model.feature_importances_, index = pred_vars).\
                  sort_values( ascending = False )
fp_randomforest.head(10)

c_gru71hd_     0.018718
e_p311t1_      0.017443
c_gru31hd_     0.016666
e_p311b_7_     0.016174
c_gru61hd_     0.016010
c_gru81hd_     0.015812
e_p311b_1_     0.015277
c_inghog1d_    0.015217
c_gashog2d_    0.014244
c_sg42_        0.013892
dtype: float64

In [35]:
# Boosted Trees
fp_xgboost = pd.Series( xgb_optimal_model.feature_importances_, index = pred_vars).\
           sort_values( ascending = False )
fp_xgboost.head(10)

c_gru52hd1_    0.035057
e_p311a5_5_    0.030793
e_p4032_       0.030132
c_ig03hd3_     0.022038
c_gru42hd1_    0.021753
e_p414_09_     0.021352
c_g05hd1_      0.019857
c_gru73hd3_    0.019725
c_ingtpu05_    0.018714
c_paesechd_    0.018075
dtype: float32